In [1]:
!!git clone https://github.com/justinpinkney/clip2latent.git


["Cloning into 'clip2latent'...",
 'remote: Enumerating objects: 404, done.\x1b[K',
 'remote: Counting objects:   1% (1/68)\x1b[K',
 'remote: Counting objects:   2% (2/68)\x1b[K',
 'remote: Counting objects:   4% (3/68)\x1b[K',
 'remote: Counting objects:   5% (4/68)\x1b[K',
 'remote: Counting objects:   7% (5/68)\x1b[K',
 'remote: Counting objects:   8% (6/68)\x1b[K',
 'remote: Counting objects:  10% (7/68)\x1b[K',
 'remote: Counting objects:  11% (8/68)\x1b[K',
 'remote: Counting objects:  13% (9/68)\x1b[K',
 'remote: Counting objects:  14% (10/68)\x1b[K',
 'remote: Counting objects:  16% (11/68)\x1b[K',
 'remote: Counting objects:  17% (12/68)\x1b[K',
 'remote: Counting objects:  19% (13/68)\x1b[K',
 'remote: Counting objects:  20% (14/68)\x1b[K',
 'remote: Counting objects:  22% (15/68)\x1b[K',
 'remote: Counting objects:  23% (16/68)\x1b[K',
 'remote: Counting objects:  25% (17/68)\x1b[K',
 'remote: Counting objects:  26% (18/68)\x1b[K',
 'remote: Counting objects:  27% (19/68)\x1

In [2]:
cd clip2latent

/content/clip2latent


In [3]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [4]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
Obtaining file:///content/clip2latent (from -r requirements.txt (line 15))
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import gradio as gr
import torch
from clip2latent import models
from PIL import Image

device = "cuda"
model_choices = {
    "faces": {
        "checkpoint": "https://huggingface.co/lambdalabs/clip2latent/resolve/main/ffhq-sg2-510.ckpt",
        "config": "https://huggingface.co/lambdalabs/clip2latent/resolve/main/ffhq-sg2-510.yaml",
        },
    "landscape": {
        "checkpoint": "https://huggingface.co/lambdalabs/clip2latent/resolve/main/lhq-sg3-410.ckpt",
        "config": "https://huggingface.co/lambdalabs/clip2latent/resolve/main/lhq-sg3-410.yaml",
    }
}

model_cache = {}
for k, v in model_choices.items():
    checkpoint = v["checkpoint"]
    cfg_file = v["config"]
    # Moving to the cpu seems to break the model, so just put all on the gpu
    model_cache[k] = models.Clip2StyleGAN(cfg_file, device, checkpoint)

@torch.no_grad()
def infer(prompt, model_select, n_samples, scale):
    model = model_cache[model_select]
    images, _ = model(prompt, n_samples_per_txt=n_samples, cond_scale=scale, skips=250, clip_sort=True)
    images = images.cpu()
    make_im = lambda x: (255*x.clamp(-1, 1)/2 + 127.5).to(torch.uint8).permute(1,2,0).numpy()
    images = [Image.fromarray(make_im(x)) for x in images]
    return images


css = """
        a {
            color: inherit;
            text-decoration: underline;
        }
        .gradio-container {
            font-family: 'IBM Plex Sans', sans-serif;
        }
        .gr-button {
            color: white;
            border-color: #9d66e5;
            background: #9d66e5;
        }
        input[type='range'] {
            accent-color: #9d66e5;
        }
        .dark input[type='range'] {
            accent-color: #dfdfdf;
        }
        .container {
            max-width: 730px;
            margin: auto;
            padding-top: 1.5rem;
        }
        #gallery {
            min-height: 22rem;
            margin-bottom: 15px;
            margin-left: auto;
            margin-right: auto;
            border-bottom-right-radius: .5rem !important;
            border-bottom-left-radius: .5rem !important;
        }
        #gallery>div>.h-full {
            min-height: 20rem;
        }
        .details:hover {
            text-decoration: underline;
        }
        .gr-button {
            white-space: nowrap;
        }
        .gr-button:focus {
            border-color: rgb(147 197 253 / var(--tw-border-opacity));
            outline: none;
            box-shadow: var(--tw-ring-offset-shadow), var(--tw-ring-shadow), var(--tw-shadow, 0 0 #0000);
            --tw-border-opacity: 1;
            --tw-ring-offset-shadow: var(--tw-ring-inset) 0 0 0 var(--tw-ring-offset-width) var(--tw-ring-offset-color);
            --tw-ring-shadow: var(--tw-ring-inset) 0 0 0 calc(3px var(--tw-ring-offset-width)) var(--tw-ring-color);
            --tw-ring-color: rgb(191 219 254 / var(--tw-ring-opacity));
            --tw-ring-opacity: .5;
        }
        #advanced-options {
            margin-bottom: 20px;
        }
        .footer {
            margin-bottom: 45px;
            margin-top: 35px;
            text-align: center;
            border-bottom: 1px solid #e5e5e5;
        }
        .footer>p {
            font-size: .8rem;
            display: inline-block;
            padding: 0 10px;
            transform: translateY(10px);
            background: white;
        }
        .dark .logo{ filter: invert(1); }
        .dark .footer {
            border-color: #303030;
        }
        .dark .footer>p {
            background: #0b0f19;
        }
        .acknowledgments h4{
            margin: 1.25em 0 .25em 0;
            font-weight: bold;
            font-size: 115%;
        }
"""



def main():
    block = gr.Blocks(css=css)

    with block:
        
        with gr.Group():
            with gr.Box():
                with gr.Row().style(mobile_collapse=False, equal_height=True):
                    text = gr.Textbox(
                        label="Enter your prompt",
                        show_label=False,
                        max_lines=1,
                        placeholder="Enter your prompt",
                    ).style(
                        border=(True, False, True, True),
                        rounded=(True, False, False, True),
                        container=False,
                    )
                    btn = gr.Button("Generate image").style(
                        margin=False,
                        rounded=(False, True, True, False),
                    )

            gallery = gr.Gallery(
                label="Generated images", show_label=False, elem_id="gallery"
            ).style(grid=[2], height="auto")


            with gr.Row(elem_id="advanced-options"):
                model_select = gr.Dropdown(label="Model", choices=["faces", "landscape"], value="faces",)
                samples = gr.Slider(label="Images", minimum=1, maximum=4, value=2, step=1)
                scale = gr.Slider(
                    label="Guidance Scale", minimum=0, maximum=10, value=2, step=0.5
                )


            

            text.submit(infer, inputs=[text, model_select, samples, scale], outputs=gallery)
            btn.click(infer, inputs=[text, model_select, samples, scale], outputs=gallery)
            

    block.queue()
    block.launch()


if __name__ == "__main__":
    main()

100%|████████████████████████████████████████| 354M/354M [00:02<00:00, 145MiB/s]
Downloading: "https://huggingface.co/lambdalabs/clip2latent/resolve/main/ffhq-sg2-510.ckpt" to /root/.cache/torch/hub/checkpoints/ffhq-sg2-510.ckpt


  0%|          | 0.00/373M [00:00<?, ?B/s]

Downloading: "https://huggingface.co/lambdalabs/clip2latent/resolve/main/lhq-sg3-410.ckpt" to /root/.cache/torch/hub/checkpoints/lhq-sg3-410.ckpt


  0%|          | 0.00/373M [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fe9f3996ef9d3faf.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://www.huggingface.co/spaces
